# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch

In [2]:
x = torch.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [5]:
import syft

In [6]:
hook = syft.TorchHook(torch)

In [7]:
torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [8]:
bob = syft.VirtualWorker(hook, id="bob")
bob.clear_objects();

In [9]:
bob._objects

{}

In [10]:
x = torch.tensor([1,2,3,4,5])

In [11]:
x = x.send(bob)

In [12]:
bob._objects

{73264147253: tensor([1, 2, 3, 4, 5])}

In [13]:
x.location

<VirtualWorker id:bob #tensors:1>

In [14]:
x.id_at_location

73264147253

In [15]:
x.id

4839848057

In [16]:
x.owner

<VirtualWorker id:me #tensors:0>

In [17]:
hook.local_worker

<VirtualWorker id:me #tensors:0>

In [18]:
x

(Wrapper)>[PointerTensor | me:4839848057 -> bob:73264147253]

In [19]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [20]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [21]:
alice = syft.VirtualWorker(hook, id='alice')
bob.clear_objects(), alice.clear_objects();

In [22]:
x = torch.randn(5)
x

tensor([ 1.9301,  0.6197, -0.0287,  0.6910,  1.4841])

In [23]:
x_pointer = x.send(bob, alice)
x_pointer

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:83307678302 -> bob:70338743424]
	-> (Wrapper)>[PointerTensor | me:30856765790 -> alice:84117444290]

In [24]:
bob._objects, alice._objects

({70338743424: tensor([ 1.9301,  0.6197, -0.0287,  0.6910,  1.4841])},
 {84117444290: tensor([ 1.9301,  0.6197, -0.0287,  0.6910,  1.4841])})

In [25]:
x_pointer.get()

[tensor([ 1.9301,  0.6197, -0.0287,  0.6910,  1.4841]),
 tensor([ 1.9301,  0.6197, -0.0287,  0.6910,  1.4841])]

In [26]:
bob._objects, alice._objects

({}, {})

In [27]:
torch.tensor([0, 1, 2, 3, 4, 5]).send(bob, alice).get(sum_results=True)

tensor([ 0,  2,  4,  6,  8, 10])

# Lesson: Introducing Remote Arithmetic

In [28]:
bob.clear_objects();

In [29]:
x = torch.tensor([1,2,3,4,5]).send(bob)
y = torch.tensor([1,1,1,1,1]).send(bob)

In [30]:
x

(Wrapper)>[PointerTensor | me:60615414665 -> bob:16866978898]

In [31]:
y

(Wrapper)>[PointerTensor | me:20829420385 -> bob:95400766976]

In [32]:
bob._objects

{16866978898: tensor([1, 2, 3, 4, 5]), 95400766976: tensor([1, 1, 1, 1, 1])}

In [33]:
z = x + y

In [34]:
z

(Wrapper)>[PointerTensor | me:88852538228 -> bob:59935378924]

In [35]:
bob._objects

{16866978898: tensor([1, 2, 3, 4, 5]),
 95400766976: tensor([1, 1, 1, 1, 1]),
 59935378924: tensor([2, 3, 4, 5, 6])}

In [36]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [37]:
z = torch.add(x,y)
z

(Wrapper)>[PointerTensor | me:4281680672 -> bob:64695180382]

In [38]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [39]:
x = torch.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = torch.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [40]:
z = (x + y).sum()

In [41]:
z.backward()

(Wrapper)>[PointerTensor | me:56189332255 -> bob:59621290374]

In [42]:
x = x.get()

In [43]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [44]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [45]:
bob.clear_objects();

In [46]:
# Remote Toy Data
toy_inputs  = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]]).send(bob)
toy_targets = torch.tensor([0, 2, 4, 6]).send(bob)

In [47]:
# Remote Weights
weights = torch.randn(2, 1, dtype=torch.float, requires_grad=True).send(bob)

In [48]:
# Remote Training
lr       = 0.5
n_epochs = 10

for i_epoch in range(n_epochs):
    preds = torch.mm(toy_inputs.float(), weights)
    mse_loss = torch.mean(torch.pow(torch.sub(toy_targets.float(), preds.view(-1)), 2))
    mse_loss.backward()
    weights.data.sub_(lr * weights.grad)
    torch.zero_(weights.grad)
    print("Epoch {:d} - MSE Loss: {:.4f}".format(i_epoch, mse_loss.get().item()))

Epoch 0 - MSE Loss: 20.3193
Epoch 1 - MSE Loss: 1.5603
Epoch 2 - MSE Loss: 0.2609
Epoch 3 - MSE Loss: 0.1082
Epoch 4 - MSE Loss: 0.0584
Epoch 5 - MSE Loss: 0.0327
Epoch 6 - MSE Loss: 0.0184
Epoch 7 - MSE Loss: 0.0103
Epoch 8 - MSE Loss: 0.0058
Epoch 9 - MSE Loss: 0.0033


In [49]:
# Get the Predictions and MSE Loss
with torch.no_grad():
    preds    = torch.mm(toy_inputs.float(), weights).view(-1)
    mse_loss = torch.mean(torch.pow(torch.sub(toy_targets.float(), preds), 2))
preds, mse_loss = preds.get(), mse_loss.get()

print("Predictions:", preds)
print("    Targets:", toy_targets.float().get())
print()
print("   MSE Loss:", mse_loss)

Predictions: tensor([0.0000, 2.0607, 3.9393, 6.0000])
    Targets: tensor([0., 2., 4., 6.])

   MSE Loss: tensor(0.0018)


# Lesson: Garbage Collection and Common Errors


In [50]:
# Clearing all objects from Bob
bob = bob.clear_objects()

In [51]:
# All the objects are gone
bob._objects

{}

In [52]:
x = torch.tensor([1,2,3,4,5]).send(bob)

In [53]:
bob._objects

{66382672223: tensor([1, 2, 3, 4, 5])}

In [54]:
# Delete the pointer to the remote tensor
del x

In [55]:
# The tensor attached to the pointer is also deleted
bob._objects

{}

In [56]:
x = torch.tensor([1,2,3,4,5]).send(bob)

In [57]:
bob._objects

{68418149120: tensor([1, 2, 3, 4, 5])}

In [58]:
# Assign a new value to the variable `x`, which deletes the original `x``
x = "asdf"

In [59]:
# Again, the tensor attached to the pointer `x` is deleted
bob._objects

{}

In [60]:
x = torch.tensor([1,2,3,4,5]).send(bob)

In [61]:
# By doing this, Jupyter Notebook caches the reference to `x`
x

(Wrapper)>[PointerTensor | me:26625674088 -> bob:96744198561]

In [62]:
bob._objects

{96744198561: tensor([1, 2, 3, 4, 5])}

In [63]:
x = "asdf"

In [64]:
# Because of Jupyter Notebook's mechanism, the reference to the pointer still exists and the remote tensor is not deleted
bob._objects

{96744198561: tensor([1, 2, 3, 4, 5])}

In [65]:
del x

In [66]:
# Deleting `x` after reassignment does not work either
bob._objects

{96744198561: tensor([1, 2, 3, 4, 5])}

In [67]:
bob = bob.clear_objects()
bob._objects

{}

In [68]:
x = torch.tensor([1,2,3,4,5]).send(bob)

In [69]:
# This attribute triggers automatic garbage collection when the pointer is deleted.
# Notice that the value is printed via `print()` function.
# If it's displayed by simply outputting `x.child.garbage_collect_data`, the reference stays alive.
print(x.child.garbage_collect_data)

True


In [70]:
bob = bob.clear_objects()
bob._objects

{}

In [71]:
for i in range(1000):
    x = torch.tensor([1,2,3,4,5]).send(bob)

In [72]:
# (Thankfully,) There aren't THOUSANDS of pointers to intermediate tensors
bob._objects

{94322671688: tensor([1, 2, 3, 4, 5])}

In [73]:
x = torch.tensor([1,2,3,4,5]).send(bob)
y = torch.tensor([1,1,1,1,1])

In [74]:
# Error due to performing an operation on a tensor and a pointer
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:75222857322 -> bob:72922510216]
Tensor B: tensor([1, 1, 1, 1, 1])

In [75]:
x = torch.tensor([1,2,3,4,5]).send(bob)
y = torch.tensor([1,1,1,1,1]).send(alice)

In [76]:
# Error due to performing an operation on tensors on different machines
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:bob #tensors:2> while the other is on <VirtualWorker id:alice #tensors:1>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [ ]:
from torch import nn, optim

In [ ]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [ ]:
# A Toy Model
model = nn.Linear(2,1)

In [ ]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [ ]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

In [ ]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [ ]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [ ]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [ ]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [ ]:
train()

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [ ]:
bob.clear_objects()
alice.clear_objects()

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
x = x.send(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
y = x + x

In [ ]:
y

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
jon = sy.VirtualWorker(hook, id="jon")

In [ ]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x = x.get()
x

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x = x.get()
x

In [ ]:
bob._objects

In [ ]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
del x

In [ ]:
bob._objects

In [ ]:
alice._objects

# Lesson: Pointer Chain Operations

In [ ]:
bob.clear_objects()
alice.clear_objects()

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x.move(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x.remote_get()

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x.move(bob)

In [ ]:
x

In [ ]:
bob._objects

In [ ]:
alice._objects